###load_data() 는 현재 tf.data로 구성되어 있음 -> 이것을 tf.placeholder로 받아오는 예제로 변경하는 소스 코드 작성

1) tf.placeholders 각 inputs, labels, lengths에 대해서 만들기 \
2) train.inputs, label.inputs 파일 읽어서 [[문장1, 문장2,...]], [[문장1_label, 문장2_label,...]] 리스트 만들기\
3) 문장 리스트 내에 있는 문장 1 : [단어1, 단어2,...] 를 모두 단어 id로 변경 -> make_vocab_table()의 word2id dictionary를 사용할 수 있음
4) get_batch_data() 함수 작성 : training example에서 batch별로 데이터 받아오는 함수\
5) get_bastch_data내에 rank2pad process를 통해 batch 내 문장들을 padding 처리해줘야 함, label 데이터에 대해서도 padding 처리 진행\
6) train.model() 부분의 sess.run 부분에 batch_data 받은 것 각각 feed_dict 해주기\
7) build_model 부분에서 label에 대한 masking 하는 소스 추가\
8) 학습!

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
# placeholders
inputs = tf.placeholder(shape=[None, None], dtype=tf.int32, name="inputs_ph")
lenghts = tf.placeholder(shape=[None], dtype=tf.int32, name="lengths_ph")

# 둘 중에 선택
# Masking 필요한 label format [3, 10] -> label에도 padding을 붙임
labels = tf.placeholder(shape=[None, None], dtype=tf.int32, name="labels_ph")

# Masking 필요 없는 label format -> [21] -> label에 padding 안 붙이고 shape을 rank 1로 설정 (data_process 쪽에서의 처리가 필요)
labels = tf.placeholder(shape=[None], dtype=tf.int32, name="labels_ph")

## Padding 처리 되어 있지 않은 리스트 padding 처리 하기
### return 값은 rank 2짜리 paddding 처리된 리스트 -> np.array로 만들 수도 있음

In [0]:
# list inputs!
# e.g. [[1,40,2,1],[5,38],[3,12,30,10,3,600,2]...] word를 id로 바꾼 문장 리스트 (각각이 문장)
def rank_2_pad_process(inputs, special_id=False):
	append_id = 0
	if special_id:
		append_id = -1 # user_id -> -1

	max_sent_len = 0
	for sent in inputs:
		max_sent_len = max(len(sent), max_sent_len)

	# print("text_a max_lengths in a batch", max_sent_len)
	padded_result = []
	sent_buffer = []
	for sent in inputs:
		for i in range(max_sent_len - len(sent)):
			sent_buffer.append(append_id)
		sent.extend(sent_buffer)
		padded_result.append(sent)
		sent_buffer = []
    
    #padded_result = np.array(padded_result)
    
	return padded_result


## File 읽어서 문장, label 별로 list 만들기

In [0]:
import os
file_path = "./CoNLL-2003"

def get_training_examples():
  _read_data(os.path.join("train.inputs"))

  with open(os.path.join(file_path, "train.inputs"), "r", encoding="utf-8") as f_handle:
  train_data = [for line in f_handle if len(line.rstrip()) > 0]


In [0]:
class InputExample(object):
	"""A single training/test example for simple sequence classification."""

	def __init__(self, input_text, label):
		self.input_text = input_text
		self.label = label

In [0]:
	def _read_data(self, data_dir, shuffle=False):
		print("[Reading %s]" % data_dir)
		with open(data_dir, "r", encoding="utf-8") as fr_handle:
			total_data = [line.strip() for line in fr_handle if len(line.strip()) > 1]
      
      split_data_list = []
      for data in total_data:
        split_data_list.append(data.split(" "))

        if shuffle and self.hparams.training_shuffle_num > 1:
			total_data = self._data_shuffling(total_data, self.hparams.training_shuffle_num)

		return total_data

	def _create_examples(self, inputs, labels, set_type):
		"""Creates examples for the training and dev sets."""
		examples = []
		for (i, input_data) in enumerate(inputs):
			

			examples.append(
				InputExample(input_text=input_text, label=labels))
		print("%s data creation is finished! %d" % (set_type, len(examples)))

		return examples

## 데이터 Batch 별로 읽어올 수 있는 부분

In [0]:

	def get_batch_data(self, curr_index, batch_size, set_type="train"):
		inputs = []
		lengths = []
		label_ids = []

		examples = {
			"train": self.train_example,
			"test": self.test_example
		}
		example = examples[set_type]

		for index, each_example in enumerate(example[curr_index * batch_size:batch_size * (curr_index + 1)]):
			tokenized_inputs_id, input_length, label_id = \
				convert_single_example(each_example, self.label_list, word2id=self.word2id)

			inputs.append(tokenized_inputs_id)
			lengths.append(input_length)
			label_ids.append(label_id)

		pad_inputs = rank_2_pad_process(inputs)

		return pad_inputs, lengths, label_ids

## 추후에 Glove pre-trained embedding을 가져다 쓸 때 필요한 부분

In [0]:
def get_word_embeddings(self):
  with np.load(self.hparams.glove_embedding_path) as data:
    print("glove embedding shape", np.shape(data["embeddings"]))
    return data["embeddings"]

In [0]:
def export_trimmed_glove_vectors():
	glove_embedding = dict()
	glove_vocab = set()

	print("Making glove trimmed.npz, It will take few minutes...")
	with open(os.path.join(glove_dir, "glove.6B.300d.txt"), "r", encoding='utf-8') as f_handle:
		for line_idx, line in enumerate(f_handle):
			line = line.strip().split(" ")
			glove_word = line[0]
			glove_vocab.add(glove_word)
			glove_embedding[glove_word] = [float(x) for x in line[1:]]

		print("# glove vocab : ", len(glove_embedding))

	with open(os.path.join(data_dir, "vocab.txt"), "r") as f_handle:
		total_vocab = [line.strip() for line in list(f_handle)]
		print("# total vocab : ", len(total_vocab))

	# print("%d words are in Glove..." % len(glove_vocab.intersection(set(total_vocab))))

	embeddings = np.zeros([len(total_vocab) + 1, embedding_dim])

	for word_idx, word in enumerate(total_vocab):
		try:
			embeddings[word_idx] = np.asarray(glove_embedding[word.lower()])
		except KeyError:
			embeddings[word_idx] = np.random.uniform(-1, 1, embedding_dim)

	np.savez_compressed(os.path.join(glove_dir, "glove.6B.300d.trimmed.npz"), embeddings=embeddings)
	print("Save Glove Embeddings as an Numpy Array : ", len(embeddings))
